# Логистическая регрессия

В этом задании вам предлагается реализовать классификатор логистическая регрессия и обучить его на датасете Large Movie Review. 

Сначала импортируем все необходимые для работы пакеты.

In [ ]:
import random
import os
from string import punctuation
from collections import defaultdict
from copy import deepcopy
import re
import numpy as np
import scipy
import scipy.sparse
import math
from __future__ import division
import time
from matplotlib import pyplot as plt

## 1. Импорт датасета, его предобработка и токенизация, построение словаря

Загружаем данные. Как и в **Assignment 1**, у нас есть три выборки: обучающая (train), размеченная тестовая (dev) и неразмеченная тестовая (test). На размеченной тестовой выборке мы тестируем классификатор и определяем его точность. Результаты работы классификатора на неразмеченной тестовой выборке проверяются на сервере СompAI, метки для нее не предоставляются.

In [ ]:
data_dir = '../FILIMDB'

def load_data(file_name):
    """
    Reads specified file, returns list of strings
    :param file_name: file name in data_dir folder
    :returns list of strings
    """
    print('Loading %s' % file_name)
    data_path = os.path.join(data_dir, file_name)
    with open(data_path) as input_data:
        lines = input_data.readlines()
        lines = [l.strip() for l in lines]
    
    print('Loaded %d lines' % len(lines))
    return lines

train_texts, train_labels = load_data('train.texts'), load_data('train.labels')
dev_texts, dev_labels = load_data('dev.texts'), load_data('dev.labels')
test_texts = load_data('test.texts')

Переведем разметку из 'pos','neg' в 1 и 0:

In [ ]:
def transform_labels(labels_str):
    labels_int = np.array([int(label == 'pos') for label in labels_str])
    return np.reshape(labels_int, (len(labels_int),1) )

In [ ]:
train_labels =transform_labels(train_labels)
dev_labels = transform_labels(dev_labels)

Выполним основную предобработку датасета:
- Отделить пунктуацию от слов пробелами
- Вычистить все лишнее: специальные символы, остатки html-разметки и т.д. При этом обычно не надо вычищать буквы с акцентами (à, á...). 'Мусорные' символы лучше не удалять, а заменять на пробелы, иначе слова могут склеиться.
- Перевести буквы в нижний регистр

In [ ]:
def tokenize(s, lower=True):
    s = re.sub(r'([^ \w])', r' \1 ',s)
    s = re.sub(r' +', r' ',s)
    if lower:
        s = s.lower()
    return s.strip().split(' ')

tokenized_train_texts = [tokenize(r) for r in train_texts]
tokenized_dev_texts = [tokenize(r) for r in dev_texts]
tokenized_test_texts = [tokenize(r) for r in test_texts]

# sanity check
assert all(len(x) > 0 for x in tokenized_train_texts)

Давайте посмотрим на результат процедуры токенизации на случайном отзыве

In [ ]:
idx = np.random.randint(0,len(tokenized_train_texts)) #choosing random index
print("Это исходный отзыв: {}".format(train_texts[idx]))
print("Это отзыв после предобработки и токенизации: {}".format(tokenized_train_texts[idx]))

Построим словарь уникальных слов, которые встречаются в обучающей выборке. Удалите слова, которые встречаются в словаре менее 5 раз.

In [ ]:
vocabulary = {}
# YOUR CODE HERE
raise NotImplementedError()

Для дальнейшей удобный работы преобразуем наш словарь в Python тип данных dict, где ключи - это наш словарь уникальные слова, а порядковый номер этого слова в сортированном по алфавиту словаре. 

In [ ]:
def transform_vocabulary(unique_words):
    # YOUR CODE HERE
    raise NotImplementedError()
    return vocabulary

In [ ]:
vocabulary = transform_vocabulary(...)

## 2. Векторизация отзывов, построение матрицы обучающей и тестовой выборки

Для того, чтобы обучить логистическую регрессию на нашем датасете, нам необходимо преобразовать его в матричный вид, где обучающая и тестовая выборка будут представлены в виде матриц размера $N \times V$, где $N$ - количество отзывов в выборке, $V$ - количество слов в словаре, а $j$-й элемент $i$-й строки соответствует частоте появления в $i$-м отзыве $j$-го токена из словаря. Для оптимизации работы с памятью мы будем строить  матрицы типа __[scipy.sparse.csr_matrix ](http://www.scipy-lectures.org/advanced/scipy_sparse/csr_matrix.html)__ 

In [ ]:
def CountVectorizer(texts, vocabulary):
        """
            Transform a collection of texts to a matrix of token counts
            In order to be memory-efficient, the matrix of token counts 
            has a sparse representation of the counts using scipy.sparse.csr_matrix
        """
        
        j_indices = [] #indices is array of column indices
        indptr = [] # indptr points to row starts in indices and data
        values = [] #values is array of corresponding nonzero values
        indptr.append(0)
        for text in texts:
            token_counter = {}
            for token in text:
                try:
                      # YOUR CODE HERE
                    raise NotImplementedError()
                except KeyError:
                    # ignore out-of-vocabulary tokens
                    continue
                    
            j_indices.extend(token_counter.keys())
            
            values.extend(token_counter.values())
            indptr.append(len(j_indices))
        
        X = scipy.sparse.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocabulary)),
                          )
    
        return X

Для того, чтобы проверить и отладить работу функции CountVectorizer() попбробуйте построить матрицу для игрушечного датасета.

In [ ]:
toy_texts = ['abs xux abs dud', 'lul omo xux xux xux']
tokenized_toy_texts = [tokenize(r) for r in toy_texts]
unique_words_toy = set(w for text in tokenized_toy_texts for w in text)
vocabulary_toy = transform_vocabulary(unique_words_toy)
print("Словарь игрушечного датасета: {}".format(sorted(unique_words_toy)))


X_toy = CountVectorizer(tokenized_toy_texts, vocabulary_toy)
print("Матрица векторизованных отзывов игрушечного датасета имеет размер {} и равна \n{}".format(X_toy.shape, X_toy.todense()))

Убедитесь, что вы получили матрицу размера $2\times5$ равную: <br>
$$[[2\: 1\: 0\: 0\: 1] \\
[0\: 0\: 1\: 1\: 3]]$$

Теперь, когда мы проверили, что функция CountVectorizer() работает корректно, построим матричные представления для нашей обучающей и тестовых выборок.

In [ ]:
X_train = CountVectorizer(tokenized_train_texts, vocabulary)
X_dev = CountVectorizer(tokenized_dev_texts, vocabulary)
X_test = CountVectorizer(tokenized_test_texts, vocabulary)

print("Размерность матрицы для обучающей выборки равна {}".format(X_train.shape))
print("Размерность матрицы для открытой тестовой выборки равна {}".format(X_dev.shape))
print("Размерность матрицы для закрытой тестовой выборки равна {}".format(X_test.shape))

Удостоверьтесь, что размер получившихся матриц равен $(25000 \times 29204), (25000 \times 29204), (10599 \times 29204)$

Как мы и говорили в задании, для удобства мы будем использовать $w^Tx_{\{i\}}$ запись, однако для этого нам нужно присоединить к входным векторам $x_{\{i\}}$ единицу, т.е. $x_{\{i\}} := [1; x_{\{i\}}]$

In [ ]:
def concat_one(X):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
X_train = concat_one(X_train)
X_dev = concat_one(X_dev)
X_test = concat_one(X_test)

Обратите внимание, что сейчас наша обучающая выборки упорядочены, для качественного обучения нам необходимо перемешать обучающую выборку случайным образом. Напишем соответствующую функцию shuffle(X,y).

In [ ]:
def shuffle(X, y):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
X_train, train_labels = shuffle(X_train, train_labels)

## 3. Построение логистической регрессии

Для логистической регрессии нам понадобится сигмоида:

$$ sigmoid(z) = {\frac {1}{1+e^{-z}}}$$

Напишите её:


In [ ]:
def sigmoid(z):
    # YOUR CODE HERE
    raise NotImplementedError()

Инициализируем вектор параметров W. Инициализировать его можно нулями.

In [ ]:
def initialize_parameters(dim):
    # YOUR CODE HERE
    raise NotImplementedError()

А теперь напишем основную фунцию, которая будет:

    1) Вычислять результат для классификатора по формуле:
    
$$h(x) = sigmoid(x^T w)$$

    2) Вычислять функцию потерь по формуле:
    
$$ L = - \frac1{N} \sum_{i=1}^N(y_i \log h(x_i) + (1 - y_i) \log (1 - h(x_i)) + \alpha\sum_{j=1}^M(w_j)^2$$

    3) Вычислять точность, с которой работает классификатор
    
    4) Находить производные:

$$ \frac{\partial L}{\partial w_j} = \frac{1}{N}x \:(h(x)-y)^T + 2\lambda w_j$$

$$ \frac{\partial L}{\partial w_0} = \frac{1}{N} \sum_{i=1}^N (h(x_{i})-y_{(i)})$$

In [ ]:
def propagate(X, y, w, alpha):
    # YOUR CODE HERE
    raise NotImplementedError()
 
    assert (dw.shape == w.shape)
    return dw, cost, accuracy

Протестируем:

In [ ]:
toy_inputs = X_toy
toy_targets = np.zeros((2, 1))
toy_dim = 5
print("Inputs: ", toy_inputs.todense())
print("Targets: ", toy_targets)


In [ ]:
toy_grads, toy_cost, toy_acc = propagate(toy_inputs, toy_targets,(initialize_parameters(dim = toy_dim)), alpha = 1e-4)

print ("dw = " + str(toy_grads))
print ("toy_cost = " + str(toy_cost))
print ("toy_acc = " + str(toy_acc))

Удостоверьтесь, что на выходе получились следующие значения:

<table style="width:50%">
    <tr>
        <td>  ** dw **  </td>
        <td> [[ 0.5]
 [ 0.25]
 [0.25]
 [0.25]
 [1.]]</td>
    </tr>
    <tr>
        <td>  ** toy_cost **  </td>
        <td> 0.69314718056 </td>
    </tr>
    <tr>
        <td>  ** toy_acc **  </td>
        <td> 1 </td>
    </tr>

</table>

Также напишем функцию обновления весов, используя следующие формулы:

$$w = w - \lambda * dL/dW$$

При этом $\lambda$ - скорость обучения (learning rate).

In [ ]:
def update_parameters(w,dw,learning_rate):
    # YOUR CODE HERE
    raise NotImplementedError()
    return w

Протестируем:

In [ ]:
new_toy_parameters = update_parameters(initialize_parameters(dim = toy_dim),toy_grads,0.01)
w = new_toy_parameters
print ("w = " + str(w))

Удостоверьтесь, что на выходе получились следующие значения:

<table style="width:50%">
    <tr>
        <td>  ** w **  </td>
        <td> [[ -0.005 ]
 [ -0.0025]
 [ -0.0025]
 [ -0.0025]
 [ -0.01]]</td>
    </tr>
   
</table>

Также напишем функцию предсказания меток $\hat{y}$ для заданных параметров w и выборки X размера $N \times V$:

In [ ]:
def predict(w, X):    
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return y_pred

In [ ]:
y_predict_toy = predict(w, toy_inputs)
print(y_predict_toy)

Удостоверьтесь, что на выходе получились следующие значения: <br> [[ 0.] <br>
 [ 0.]]

## 3. Обучение логистической регрессии

Теперь, когда вся предварительная работа выполнена, можно начать обучение логистической регрессии. Установим значения гиперпараметров: скорость обучения, коэффициент регуляризации $\alpha$, количество итераций.

In [ ]:
N, dims = X_train.shape

learning_rate = 1e-3
alpha = 1e-5

num_epochs = 500
batch_size = 100
num_batches = np.ceil(N / batch_size).astype('int')

w = initialize_parameters(dim=dims)
acc = 0
best_acc = 0
best_parameters = None
costs = []

Начнем обучение. Обратите внимание, что для ускорения сходимости, мы будем использовать mini_batch подход.

In [ ]:
start = time.time()
for epoch in range(num_epochs):
    for batch in  range(num_batches):
        # form batch
        # YOUR CODE HERE
        raise NotImplementedError()

        # propagate
        # YOUR CODE HERE
        raise NotImplementedError()
        
    if epoch % 10 == 0:
        # propagate through whole train dataset and calculate cost, acc
        # YOUR CODE HERE
        raise NotImplementedError()
        costs.append(cost)
        print("Epoch {} out of {}, Cost: {}, Acc: {}".format(epoch,num_epochs, cost, acc))

        # calculate cost and acc on dev dataset
        # YOUR CODE HERE
        raise NotImplementedError()
        print("Test acc: {}".format(dev_acc))
        
        #save
        if dev_acc > best_acc:
            best_acc = dev_acc
            best_parameters = w
end = time.time()

In [ ]:
print("Training time: {}, Best acc: {}".format(end - start, best_acc))

Давайте нарисуем кривую обучения: cost vs iteration.

In [ ]:
plt.plot(costs)
plt.ylabel('Cost')
plt.xlabel('Iterations (per hundreds)')
plt.show()

## 4. Оптимизация гиперпараметров

Сейчас в качестве коэффициента $L_2$ регуляризации $\alpha$ мы выбрали некоторое произвольное значение. Для подбора оптимального значения $\alpha$ мы будем проводить валидацию на валидационной выборке. Для этого случайным образом перемешаем текущую обучающую выборку и разделим ее в соотношении 80:20 на две – новая (меньшего размера) обучающая выборка и валидационная выборка. 

In [ ]:
X_train, train_labels = shuffle(X_train, train_labels)
small_train_size = int(math.ceil(X_train.shape[0]*0.8))
X_small_train = X_train[:small_train_size]
train_labels_small = train_labels[:small_train_size]
X_valid =  X_train[small_train_size:]
labels_valid = train_labels[small_train_size:]
print("Размер новой обучающей выбоки равен {}, размер валидационной выборки равен {}".format(X_small_train.shape[0], X_valid.shape[0]))

Теперь попробуем найти оптимальное значение $\alpha$

In [ ]:
N, dims = X_small_train.shape

learning_rate = 1e-3
alphas = [1e-2, 5e-2, 1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]

num_epochs = 500
batch_size = 100
num_batches = np.ceil(N / batch_size).astype('int')

best_alpha = alphas[0]
best_alpha_acc = 0
best_alpha_parameters = None

cost_train = np.zeros((len(alphas), num_epochs//10))

for j, alpha in enumerate(alphas):
    print("Коэффициент регуляризации = {}".format(alpha))
    w = initialize_parameters(dim=dims)
    best_acc = 0
    best_parameters = None
      
    for epoch in range(num_epochs):
        for batch in  range(num_batches):
            # form batch
            # YOUR CODE HERE
            raise NotImplementedError()

            # propagate
            # YOUR CODE HERE
            raise NotImplementedError()
        
        if epoch % 10 == 0:
            # propagate through whole train dataset and calculate cost, acc
            # YOUR CODE HERE
            raise NotImplementedError()
            
            print("Epoch {} out of {}, Cost: {}, Acc: {}".format(epoch,num_epochs, cost, acc))

            # calculate cost and acc on dev dataset
            # YOUR CODE HERE
            raise NotImplementedError()
        
            #save
            if valid_acc > best_acc:
                best_acc = valid_acc
                best_parameters = w
            
    if best_acc > best_alpha_acc:
        best_alpha = alpha
        best_alpha_acc = best_acc
        best_alpha_parameters = best_parameters

In [ ]:
print('Оптимальное значение коэффициента регуляризации равно {} '.format(best_alpha))

Теперь используя найденное значение $\alpha$, обучим логистическую регрессию на всей обучающей выборке и оценим точность.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print("Training time: {}, Best acc: {}".format(end - start, best_acc))

## 5. Загрузка результатов на сервер

Используя найденные оптимальные параметры, посчитаем метки для всех трех датасетов

In [ ]:
w_best = best_parameters
train_preds = predict(w_best, X_train)
dev_preds = predict(w_best, X_dev)
test_preds = predict(w_best, X_test)

Создадим файл с прогнозами и отправим его на CompAI сервер. Для этого понадобится положить файл compai_ilimdb_sentiment.py, полученный при регистрации, в директорию с этим блокнотом.

In [ ]:
res_file_name = "preds.tsv"
with open(res_file_name, 'w') as outp:
    for index, label in enumerate(train_preds):
        print('train/%d\t%s' % (index, label), file=outp)
    for index, label in enumerate(dev_preds):
        print('dev/%d\t%s' % (index, label), file=outp)
    for index, label in enumerate(test_preds):
        print('test/%d\t%s' % (index, label), file=outp)
print('Predictions saved to %s' % res_file_name)

%run compai_ilimdb_sentiment.py submit $res_file_name